# Введение в анализ данных (первый семестр)
## Домашнее задание №3 - Участие в конкурсе Rossmann Store Sales

<b>Дедлайн сдачи домашнего задания: 28 апреля, 23:59. После дедлайна решения не принимаются.</b><br>
<br>
В данном домашнем задании вам придется учавствовать в конкурсе [Rossman Store Sales](https://www.kaggle.com/c/rossmann-store-sales). Ваша задача получить наиболее высокий результат на Private Leaderboard, используя только те методы, которые вы узнали до сегодняшнего дня на курсах Техносферы. Оценка домашнего задания будет производиться через kaggle kernels. До дедлайна вам необходимо прислать на почту письмо с ссылкой на ваш kernel. Не забудьте указать правильно тему:)

<h2>Подготовка</h2>

Импорт всех необходимых пакетов:

In [1]:
import gc
import operator
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, TimeSeriesSplit
import matplotlib.pyplot as plt
%matplotlib inline

Описание некоторых необходимых функций, используемых далее:

In [2]:
def to_weight(y):
    w = np.zeros(y.shape, dtype=float)
    ind = y != 0
    w[ind] = 1./(y[ind]**2)
    return w

def rmspe(y, yhat):
    w = to_weight(y)
    rmspe = np.sqrt(np.mean( w * (y - yhat)**2 ))
    return rmspe

In [3]:
def str_date_to_ordinal(date):
    return datetime.strptime(date, '%Y-%m-%d').date().toordinal()

def date_to_ordinal(date):
    return date.toordinal()

def str_to_date(date):
    return datetime.strptime(date, '%Y-%m-%d').date()

In [4]:
SEED = 42

<h2>Загрузка и предобработка данных</h2>

<h3>1) Загрузка тренировочных и тестовых данных</h3>

In [5]:
train_df = pd.read_csv('./data/train.csv', sep=',', parse_dates=['Date'], date_parser=str_to_date,
                      low_memory=False)
test_df = pd.read_csv('./data/test.csv', sep=',', parse_dates=['Date'], date_parser=str_to_date,
                      low_memory=False)

In [6]:
print train_df.shape
print "Train data head:"
train_df.head()

(1017209, 9)
Train data head:


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [7]:
print test_df.shape
print "Test data head:"
test_df.head()

(41088, 8)
Test data head:


,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
1,2,3,4,2015-09-17,1.0,1,0,0
2,3,7,4,2015-09-17,1.0,1,0,0
3,4,8,4,2015-09-17,1.0,1,0,0
4,5,9,4,2015-09-17,1.0,1,0,0


<h3>2) Обработка пропущенных значений в тренировочных и тестовых данных</h3>

In [8]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 9 columns):
Store            1017209 non-null int64
DayOfWeek        1017209 non-null int64
Date             1017209 non-null datetime64[ns]
Sales            1017209 non-null int64
Customers        1017209 non-null int64
Open             1017209 non-null int64
Promo            1017209 non-null int64
StateHoliday     1017209 non-null object
SchoolHoliday    1017209 non-null int64
dtypes: datetime64[ns](1), int64(7), object(1)
memory usage: 69.8+ MB


Как видно из информации о тренировочной выборке, пропущенных значений в ней нет.

In [9]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41088 entries, 0 to 41087
Data columns (total 8 columns):
Id               41088 non-null int64
Store            41088 non-null int64
DayOfWeek        41088 non-null int64
Date             41088 non-null datetime64[ns]
Open             41077 non-null float64
Promo            41088 non-null int64
StateHoliday     41088 non-null object
SchoolHoliday    41088 non-null int64
dtypes: datetime64[ns](1), float64(1), int64(5), object(1)
memory usage: 2.5+ MB


Как видно из информации о тестовой выборке, пропущенных значений в ней нет нигде, кроме 11 пропущенных значений переменной $Open$.

In [10]:
test_df[pd.isnull(test_df.Open)]

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
479,480,622,4,2015-09-17,NaN,1,0,0
1335,1336,622,3,2015-09-16,NaN,1,0,0
2191,2192,622,2,2015-09-15,NaN,1,0,0
3047,3048,622,1,2015-09-14,NaN,1,0,0
4759,4760,622,6,2015-09-12,NaN,0,0,0
5615,5616,622,5,2015-09-11,NaN,0,0,0
6471,6472,622,4,2015-09-10,NaN,0,0,0
7327,7328,622,3,2015-09-09,NaN,0,0,0
8183,8184,622,2,2015-09-08,NaN,0,0,0
9039,9040,622,1,2015-09-07,NaN,0,0,0


Видно, что все 11 пропущенных значений относятся к магазину 622. Предположим, что магазин был открыт ($Open$ == 1), т.к. если окажется, что магазин был закрыт, то мы предскажем $Sales$ = 0, и это никак не повлияет на ошибку в предсказании. В случае же, если магазин был на самом деле открыт, а мы предположили, что он закрыт и автоматически предсказали 0, мы получим ошибку.

In [11]:
test_df.loc[pd.isnull(test_df.Open), 'Open'] = 1

<h3>3) Обработка категориальных признаков в тренировочных и тестовых данных</h3>

Объединим выборки в одну для анализа значений категориальных переменных. Заметим, что в тестовой выборке есть переменная $Id$, не принимающая NaN значений, а в тренировочной выборке данной переменной нет. Поэтому по значению Nan или non-NaN данной переменной будем различать тренировочную и тестовую выборки.

In [12]:
all_df = pd.concat([train_df, test_df], axis=0)
all_df.head()

,Customers,Date,DayOfWeek,Id,Open,Promo,Sales,SchoolHoliday,StateHoliday,Store
0,555.0,2015-07-31,5,NaN,1.0,1,5263.0,1,0,1
1,625.0,2015-07-31,5,NaN,1.0,1,6064.0,1,0,2
2,821.0,2015-07-31,5,NaN,1.0,1,8314.0,1,0,3
3,1498.0,2015-07-31,5,NaN,1.0,1,13995.0,1,0,4
4,559.0,2015-07-31,5,NaN,1.0,1,4822.0,1,0,5


In [13]:
all_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1058297 entries, 0 to 41087
Data columns (total 10 columns):
Customers        1017209 non-null float64
Date             1058297 non-null datetime64[ns]
DayOfWeek        1058297 non-null int64
Id               41088 non-null float64
Open             1058297 non-null float64
Promo            1058297 non-null int64
Sales            1017209 non-null float64
SchoolHoliday    1058297 non-null int64
StateHoliday     1058297 non-null object
Store            1058297 non-null int64
dtypes: datetime64[ns](1), float64(4), int64(4), object(1)
memory usage: 88.8+ MB


In [14]:
unique_StateHoliday_values = np.sort(all_df.StateHoliday.unique())
print "Train data StateHoliday unique values:", np.sort(train_df.StateHoliday.unique())
print "Test data StateHoliday unique values: ", np.sort(test_df.StateHoliday.unique())
print "All data StateHoliday unique values:  ", unique_StateHoliday_values

Train data StateHoliday unique values: ['0' 'a' 'b' 'c']
Test data StateHoliday unique values:  ['0' 'a']
All data StateHoliday unique values:   ['0' 'a' 'b' 'c']


Как видно, в тестовых данных отсутствуют категории 'b' и 'c'.

Пронумеруем все уникальные значения переменной $StateHoliday$: 

In [15]:
all_df['StateHoliday'] = all_df['StateHoliday'].astype('category').cat.codes

Преобразуем переменную $Date$.

In [17]:
feature_name = 'Date'
dispatcher = {'DayOfMonth': pd.Index(all_df[feature_name]).day, 
              'WeekOfYear': pd.Index(all_df[feature_name]).week,
              'MonthOfYear': pd.Index(all_df[feature_name]).month,
              'Year': pd.Index(all_df[feature_name]).year,
              'DayOfYear': pd.Index(all_df[feature_name]).dayofyear
             }

for new_feat_suffx, mapping in dispatcher.iteritems():
    all_df[feature_name + new_feat_suffx] = mapping

all_df[feature_name] = all_df[feature_name].apply(date_to_ordinal)

all_df.head()

,Customers,Date,DayOfWeek,Id,Open,Promo,Sales,SchoolHoliday,StateHoliday,Store,DateDayOfYear,DateYear,DateMonthOfYear,DateWeekOfYear,DateDayOfMonth
0,555.0,735810,5,NaN,1.0,1,5263.0,1,0,1,212,2015,7,31,31
1,625.0,735810,5,NaN,1.0,1,6064.0,1,0,2,212,2015,7,31,31
2,821.0,735810,5,NaN,1.0,1,8314.0,1,0,3,212,2015,7,31,31
3,1498.0,735810,5,NaN,1.0,1,13995.0,1,0,4,212,2015,7,31,31
4,559.0,735810,5,NaN,1.0,1,4822.0,1,0,5,212,2015,7,31,31


In [18]:
all_df.columns

Index([u'Customers', u'Date', u'DayOfWeek', u'Id', u'Open', u'Promo', u'Sales',
       u'SchoolHoliday', u'StateHoliday', u'Store', u'DateDayOfYear',
       u'DateYear', u'DateMonthOfYear', u'DateWeekOfYear', u'DateDayOfMonth'],
      dtype='object')

<h3>4) Удаление ненужных признаков из тренировочных и тестовых данных</h3>

Выберем для удаления признаки, которые больше не нужны в выборках:

In [19]:
# if we vectorized StateHoliday feature
#columns_to_drop = ['StateHoliday']
columns_to_drop = []

Добавим линейно зависимые признаки в список признаков для удаления:

In [20]:
# if we vectorized StateHoliday feature
#columns_to_drop.append('StateHoliday_0')

Удалим выбранные признаки:

In [21]:
all_df.drop(columns_to_drop, axis=1, inplace=True)
all_df.head()

,Customers,Date,DayOfWeek,Id,Open,Promo,Sales,SchoolHoliday,StateHoliday,Store,DateDayOfYear,DateYear,DateMonthOfYear,DateWeekOfYear,DateDayOfMonth
0,555.0,735810,5,NaN,1.0,1,5263.0,1,0,1,212,2015,7,31,31
1,625.0,735810,5,NaN,1.0,1,6064.0,1,0,2,212,2015,7,31,31
2,821.0,735810,5,NaN,1.0,1,8314.0,1,0,3,212,2015,7,31,31
3,1498.0,735810,5,NaN,1.0,1,13995.0,1,0,4,212,2015,7,31,31
4,559.0,735810,5,NaN,1.0,1,4822.0,1,0,5,212,2015,7,31,31


<h3>5) Загрузка данных о магазинах</h3>

In [22]:
store_df = pd.read_csv('./data/store.csv', sep=',')
store_df.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


In [23]:
store_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1115 entries, 0 to 1114
Data columns (total 10 columns):
Store                        1115 non-null int64
StoreType                    1115 non-null object
Assortment                   1115 non-null object
CompetitionDistance          1112 non-null float64
CompetitionOpenSinceMonth    761 non-null float64
CompetitionOpenSinceYear     761 non-null float64
Promo2                       1115 non-null int64
Promo2SinceWeek              571 non-null float64
Promo2SinceYear              571 non-null float64
PromoInterval                571 non-null object
dtypes: float64(5), int64(2), object(3)
memory usage: 87.2+ KB


Видно, что дистанции до магазинов-конкурентов определены практически для всех магазинов, а это значит, что даты открытия конкурентов не могут иметь какие-либо краевые значения (т.к. конкуренты действительно у этих магазинов есть). Поэтому в будущем, при преобразовании данных признаков в дату, будем пропущенные значения заполнять средним значением.

<h3>6) Обработка категориальных признаков в данных о магазинах</h3>

Категориальные признаки - $StoreType$ и $Assortment$. Также обработаем все временные признаки.

In [24]:
unique_StoreType_values =np.sort(store_df['StoreType'].unique())
unique_Assortment_values = np.sort(store_df['Assortment'].unique())
print "Unique values of StoreType: ", unique_StoreType_values
print "Unique values of Assortment:", unique_Assortment_values

Unique values of StoreType:  ['a' 'b' 'c' 'd']
Unique values of Assortment: ['a' 'b' 'c']


In [25]:
store_df['StoreType'] = store_df['StoreType'].astype('category').cat.codes
store_df['Assortment'] = store_df['Assortment'].astype('category').cat.codes
print "Unique values of StoreType: ", np.sort(store_df['StoreType'].unique())
print "Unique values of Assortment:", np.sort(store_df['Assortment'].unique())

Unique values of StoreType:  [0 1 2 3]
Unique values of Assortment: [0 1 2]


In [26]:
#StoreType_columns_names = ["StoreType_%c" % c for c in unique_StoreType_values]
#StoreType_columns = OneHotEncoder(sparse=False).fit_transform(store_df.StoreType.reshape(-1,1))
#store_df = pd.concat([store_df, pd.DataFrame(StoreType_columns, columns=StoreType_columns_names, 
                                         #index=store_df.index)],
                   #axis=1)

#Assortment_columns_names = ["Assortment_%c" % c for c in unique_Assortment_values]
#Assortment_columns = OneHotEncoder(sparse=False).fit_transform(store_df.Assortment.reshape(-1,1))
#store_df = pd.concat([store_df, pd.DataFrame(Assortment_columns, columns=Assortment_columns_names, 
                                         #index=store_df.index)],
                   #axis=1)

#store_df.head()

Далее необходимо преобразовать переменные $CompetitionOpenSinceYear$ и $CompetitionOpenSinceMonth$ к числовому представлению, чтобы можно было сравнивать для каждой из покупок, была ли она совершена в данный период, или нет. Пропуски заполним средними значениями.

In [27]:
def CompetitionOpenSince_to_ordinal(row):
    try:
        date = '%d-%d' % (int(row['CompetitionOpenSinceYear']), int(row['CompetitionOpenSinceMonth']))
        return datetime.strptime(date, '%Y-%m').date().toordinal()
    except:
        return np.nan

In [28]:
store_df['CompetitionOpenSinceDate'] = store_df.apply(CompetitionOpenSince_to_ordinal, axis=1)
mean_CompetitionOpenSince = store_df['CompetitionOpenSinceDate'].mean()
store_df['CompetitionOpenSinceDate'] = store_df['CompetitionOpenSinceDate'].fillna(
                                        mean_CompetitionOpenSince).astype(np.int64)
store_df.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,CompetitionOpenSinceDate
0,1,2,0,1270.0,9.0,2008.0,0,NaN,NaN,NaN,733286
1,2,0,0,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct",732981
2,3,0,0,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct",732646
3,4,2,2,620.0,9.0,2009.0,0,NaN,NaN,NaN,733651
4,5,0,0,29910.0,4.0,2015.0,0,NaN,NaN,NaN,735689


Теперь необходимо преобразовать переменные $Promo2SinceYear$ и $Promo2SinceWeek$ числовому представлению, чтобы можно было сравнивать для каждой из покупок, была ли она совершена в данный период, или нет. Для этого введём переменную $Promo2SinceDate$. Если $Promo2SinceYear$ и $Promo2SinceWeek$ заданы для магазина, то $Promo2SinceDate$ = date($Promo2SinceYear$, $Promo2SinceWeek$).toordinal(), иначе  $Promo2SinceDate$ = 0 для магазинов с $Promo2$ == 0, $Promo2SinceDate$ = среднему для магазинов с $Promo2$ == 1.

In [29]:
def Promo2Since_to_ordinal(row):
    try:
        date = '%d-W%d' % (int(row['Promo2SinceYear']), int(row['Promo2SinceWeek']))
        return datetime.strptime(date + '-1', '%Y-W%W-%w').date().toordinal()
    except:
        return np.nan

In [30]:
store_df['Promo2SinceDate'] = store_df.apply(Promo2Since_to_ordinal, axis=1)
mean_Promo2SinceDate = store_df['Promo2SinceDate'].mean()
store_df['Promo2SinceDate'] = store_df['Promo2SinceDate'].fillna(mean_Promo2SinceDate).astype(np.int64)
store_df.loc[store_df['Promo2'] == 0, 'Promo2SinceDate'] = 0
store_df.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,CompetitionOpenSinceDate,Promo2SinceDate
0,1,2,0,1270.0,9.0,2008.0,0,NaN,NaN,NaN,733286,0
1,2,0,0,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct",732981,733860
2,3,0,0,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct",732646,734231
3,4,2,2,620.0,9.0,2009.0,0,NaN,NaN,NaN,733651,0
4,5,0,0,29910.0,4.0,2015.0,0,NaN,NaN,NaN,735689,0


И, наконец, необходимо преобразовать переменную $PromoInterval$: будем хранить кортеж номеров месяцев,  которые проходят сезонные распродажи, либо мода, если значение пропущенное.

In [31]:
store_df['PromoInterval'].unique()

array([nan, 'Jan,Apr,Jul,Oct', 'Feb,May,Aug,Nov', 'Mar,Jun,Sept,Dec'], dtype=object)

In [32]:
mode_PromoInterval = store_df[store_df['PromoInterval'].notnull()]['PromoInterval'].mode()[0]
promo_intervals = {'Jan,Apr,Jul,Oct': (1,4,7,10), 
                   'Feb,May,Aug,Nov': (2,5,8,11), 
                   'Mar,Jun,Sept,Dec': (3,6,9,12)
                  } # ()}
promo_intervals[np.nan] = promo_intervals[mode_PromoInterval]
store_df['PromoInterval'] = store_df['PromoInterval'].apply(lambda x: promo_intervals[x])
store_df.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,CompetitionOpenSinceDate,Promo2SinceDate
0,1,2,0,1270.0,9.0,2008.0,0,NaN,NaN,"(1, 4, 7, 10)",733286,0
1,2,0,0,570.0,11.0,2007.0,1,13.0,2010.0,"(1, 4, 7, 10)",732981,733860
2,3,0,0,14130.0,12.0,2006.0,1,14.0,2011.0,"(1, 4, 7, 10)",732646,734231
3,4,2,2,620.0,9.0,2009.0,0,NaN,NaN,"(1, 4, 7, 10)",733651,0
4,5,0,0,29910.0,4.0,2015.0,0,NaN,NaN,"(1, 4, 7, 10)",735689,0


<h3>7) Удаление ненужных признаков из данных о магазинах</h3>

Выберем переменные, которые можно удалить из датасета с информацией о магазинах из-за ненадобности:

In [33]:
columns_to_drop = ['CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear',
                   'Promo2SinceWeek', 'Promo2SinceYear']

Удалим все выбранные столбцы из датасета с информацией о магазинах:

In [34]:
store_df.drop(columns_to_drop, axis=1, inplace=True)
store_df.head()

,Store,StoreType,Assortment,CompetitionDistance,Promo2,PromoInterval,CompetitionOpenSinceDate,Promo2SinceDate
0,1,2,0,1270.0,0,"(1, 4, 7, 10)",733286,0
1,2,0,0,570.0,1,"(1, 4, 7, 10)",732981,733860
2,3,0,0,14130.0,1,"(1, 4, 7, 10)",732646,734231
3,4,2,2,620.0,0,"(1, 4, 7, 10)",733651,0
4,5,0,0,29910.0,0,"(1, 4, 7, 10)",735689,0


<h3>8) Обработка пропущенных значений в данных о магазинах</h3>

Во время обработки категориальных и временных признаков мы обработали часть пропущенных значений. Обработаем оставшиеся пропущенные значения в получившейся выборке.

In [35]:
store_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1115 entries, 0 to 1114
Data columns (total 8 columns):
Store                       1115 non-null int64
StoreType                   1115 non-null int8
Assortment                  1115 non-null int8
CompetitionDistance         1112 non-null float64
Promo2                      1115 non-null int64
PromoInterval               1115 non-null object
CompetitionOpenSinceDate    1115 non-null int64
Promo2SinceDate             1115 non-null int64
dtypes: float64(1), int64(4), int8(2), object(1)
memory usage: 54.5+ KB


Пропущенные значения для признака $CompetitionDistance$ заполним средним значением:

In [36]:
mean_CompetitionDistance = store_df.CompetitionDistance.mean()
store_df['CompetitionDistance'] = store_df['CompetitionDistance'].fillna(mean_CompetitionDistance)

In [37]:
store_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1115 entries, 0 to 1114
Data columns (total 8 columns):
Store                       1115 non-null int64
StoreType                   1115 non-null int8
Assortment                  1115 non-null int8
CompetitionDistance         1115 non-null float64
Promo2                      1115 non-null int64
PromoInterval               1115 non-null object
CompetitionOpenSinceDate    1115 non-null int64
Promo2SinceDate             1115 non-null int64
dtypes: float64(1), int64(4), int8(2), object(1)
memory usage: 54.5+ KB


<h3>9) Слияние данных о продажах и о магазинах</h3>

In [38]:
df = pd.merge(all_df, store_df, how='left', on=['Store'])
df.head().T

,0,1,2,3,4
Customers,555,625,821,1498,559
Date,735810,735810,735810,735810,735810
DayOfWeek,5,5,5,5,5
Id,NaN,NaN,NaN,NaN,NaN
Open,1,1,1,1,1
Promo,1,1,1,1,1
Sales,5263,6064,8314,13995,4822
SchoolHoliday,1,1,1,1,1
StateHoliday,0,0,0,0,0
Store,1,2,3,4,5


In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1058297 entries, 0 to 1058296
Data columns (total 22 columns):
Customers                   1017209 non-null float64
Date                        1058297 non-null int64
DayOfWeek                   1058297 non-null int64
Id                          41088 non-null float64
Open                        1058297 non-null float64
Promo                       1058297 non-null int64
Sales                       1017209 non-null float64
SchoolHoliday               1058297 non-null int64
StateHoliday                1058297 non-null int8
Store                       1058297 non-null int64
DateDayOfYear               1058297 non-null int32
DateYear                    1058297 non-null int32
DateMonthOfYear             1058297 non-null int32
DateWeekOfYear              1058297 non-null int32
DateDayOfMonth              1058297 non-null int32
StoreType                   1058297 non-null int8
Assortment                  1058297 non-null int8
CompetitionDistan

<h3>10) Введение новых признаков для учёта промоакций и конкурентов</h3>

Введём 3 новых переменных:<br>
1) $Promo2Today$ - есть ли сегодня сезонная распродажа в этом магазине<br>
2) $CompetitionIsOpen$ - открыт ли сегодня какой-либо конкурент этого магазина<br>
3) $AverageCheck$ - средний чек для этого магазина за всё время<br>

In [40]:
def is_promo2_today(row):
    return int(row['Promo2'] == 1 and row['Promo2SinceDate'] <= row['Date'] \
           and row['DateMonthOfYear'] in row['PromoInterval'])

def is_competition_open(row):
    return int(row['CompetitionOpenSinceDate'] <= row['Date'])

In [41]:
df['Promo2Today'] = df.apply(is_promo2_today, axis=1)
df['CompetitionIsOpen'] = df.apply(is_competition_open, axis=1)
avg_checks = pd.DataFrame(df.groupby('Store')['Sales'].sum().astype(np.float64) \
             / df.groupby('Store')['Customers'].sum().astype(np.float64), 
                          columns=['AverageCheck']).reset_index()
df = df.merge(avg_checks, on='Store', how='left')
df.head().T

,0,1,2,3,4
Customers,555,625,821,1498,559
Date,735810,735810,735810,735810,735810
DayOfWeek,5,5,5,5,5
Id,NaN,NaN,NaN,NaN,NaN
Open,1,1,1,1,1
Promo,1,1,1,1,1
Sales,5263,6064,8314,13995,4822
SchoolHoliday,1,1,1,1,1
StateHoliday,0,0,0,0,0
Store,1,2,3,4,5


Удаление столбцов, не нужных для применения модели:

In [ ]:
# TODO: delete numeric dates, compare with model with them
columns_to_drop = ['PromoInterval', 'Date', 'CompetitionOpenSinceDate', 'Promo2SinceDate']
df.drop(columns_to_drop, axis=1, inplace=True)

<h2>Применение Random Forest Regressor для предсказания продаж</h2>

Будем анализировать только открытые магазины (закрытый магазин должен приносить прибыль 0):

In [43]:
train_df = df[(df['Id'].isnull()) & (df['Open'] == 1)].drop(['Id'], axis=1)
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 844392 entries, 0 to 1017190
Data columns (total 20 columns):
Customers              844392 non-null float64
DayOfWeek              844392 non-null int64
Open                   844392 non-null float64
Promo                  844392 non-null int64
Sales                  844392 non-null float64
SchoolHoliday          844392 non-null int64
StateHoliday           844392 non-null int8
Store                  844392 non-null int64
DateDayOfYear          844392 non-null int32
DateYear               844392 non-null int32
DateMonthOfYear        844392 non-null int32
DateWeekOfYear         844392 non-null int32
DateDayOfMonth         844392 non-null int32
StoreType              844392 non-null int8
Assortment             844392 non-null int8
CompetitionDistance    844392 non-null float64
Promo2                 844392 non-null int64
Promo2Today            844392 non-null int64
CompetitionIsOpen      844392 non-null int64
AverageCheck           84439

In [44]:
test_df = df[df['Id'].notnull()].drop(['Sales', 'Customers'], axis=1)
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41088 entries, 1017209 to 1058296
Data columns (total 19 columns):
DayOfWeek              41088 non-null int64
Id                     41088 non-null float64
Open                   41088 non-null float64
Promo                  41088 non-null int64
SchoolHoliday          41088 non-null int64
StateHoliday           41088 non-null int8
Store                  41088 non-null int64
DateDayOfYear          41088 non-null int32
DateYear               41088 non-null int32
DateMonthOfYear        41088 non-null int32
DateWeekOfYear         41088 non-null int32
DateDayOfMonth         41088 non-null int32
StoreType              41088 non-null int8
Assortment             41088 non-null int8
CompetitionDistance    41088 non-null float64
Promo2                 41088 non-null int64
Promo2Today            41088 non-null int64
CompetitionIsOpen      41088 non-null int64
AverageCheck           41088 non-null float64
dtypes: float64(4), int32(5), int64(7), in

In [45]:
X_train = train_df[train_df.columns.drop(['Customers', 'Sales'])].values
y_train = train_df['Sales'].values
print X_train.shape, y_train.shape

(844392L, 18L) (844392L,)


In [46]:
X_test = test_df[test_df.columns.drop(['Id'])].values
X_test.shape

(41088L, 18L)

Выбираем тренировочную и тестовую подвыборки небольшого размера:

In [47]:
#X_train_train, X_train_test, y_train_train, y_train_test = train_test_split(X_train, y_train,
                                                                           #train_size=0.08,
                                                                           #test_size=0.02,
                                                                           #random_state=SEED)
#print X_train_train.shape, X_train_test.shape, y_train_train.shape, y_train_test.shape

In [48]:
columns = train_df.columns.drop(['Sales', 'Customers'])
del train_df

In [56]:
gc.collect()

379

Подбираем наилучшие параметры модели с помощью поиска по сетке параметров с использованием кросс-валидации на 5 разбиениях временного ряда:

In [55]:
'''
param_grid = {'n_estimators': (10, 50, 80, 100),
              'criterion': ('mse',),
              'max_depth': (5, 10, 15, 20, None)}
best_n_estimators = None
best_criterion = None
best_max_depth = None
best_rsmpe_score = 100.0
for criterion in param_grid['criterion']:
    for n_estimators in param_grid['n_estimators']:
        for max_depth in param_grid['max_depth']:
            print 'n_estimators =', n_estimators, 'criterion =', criterion, 'max_depth =', max_depth
            scores = []
            tss = TimeSeriesSplit(n_splits=5)
            for train, test in tss.split(X_train_train):
                rf_model = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth,
                                                 criterion=criterion, random_state=SEED, n_jobs=1)
                rf_model.fit(X_train_train[train], y_train_train[train])
                scores.append(rmspe(y_train_train[test], rf_model.predict(X_train_train[test])))
                del rf_model
                gc.collect()
            cur_rsmpe_score = np.mean(np.array(scores))
            print 'rsmpe_score =', cur_rsmpe_score
            if cur_rsmpe_score < best_rsmpe_score:
                best_rsmpe_score = cur_rsmpe_score
                best_n_estimators = n_estimators
                best_criterion = criterion
                best_max_depth = max_depth
            del scores, tss
            gc.collect()
            print '----------------------------------------------------------'
        
print 'best n_estimators:', best_n_estimators, 'best criterion:', best_criterion
print 'rmspe for best params:', best_rsmpe_score
'''

n_estimators = 10 criterion = mse max_depth = 5
rsmpe_score = 0.444912158098
----------------------------------------------------------
n_estimators = 10 criterion = mse max_depth = 10
rsmpe_score = 0.380676726535
----------------------------------------------------------
n_estimators = 10 criterion = mse max_depth = 15
rsmpe_score = 0.296350019077
----------------------------------------------------------
n_estimators = 10 criterion = mse max_depth = 20
rsmpe_score = 0.239696414616
----------------------------------------------------------
n_estimators = 10 criterion = mse max_depth = None
rsmpe_score = 0.220703127173
----------------------------------------------------------
n_estimators = 50 criterion = mse max_depth = 5
rsmpe_score = 0.444539510519
----------------------------------------------------------
n_estimators = 50 criterion = mse max_depth = 10
rsmpe_score = 0.378287301831
----------------------------------------------------------
n_estimators = 50 criterion = mse max_dep

best n_estimators: 100<br>

Обучим модель с наилучшими найденными параметрами на всех тестовых данных, взяв количество деревьев равным 50, т.к. оценка для 50 деревьев почти не отличается от оценки для 100, а время обучения модели для 50 деревьев меньше:

In [50]:
rf = RandomForestRegressor(n_estimators=50, random_state=SEED)
rf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=50, n_jobs=1, oob_score=False, random_state=42,
           verbose=0, warm_start=False)

In [ ]:
#print rmspe(y_train_test, rf.predict(X_train_test))

In [51]:
test_predicted = rf.predict(X_test)
test_predicted.shape

(41088L,)

In [71]:
submission = pd.concat([test_df.Id.astype(int), pd.DataFrame(test_predicted, 
                                                 columns=['Sales'], index=test_df.index)], axis=1)
submission.head()

,Id,Sales
1017209,1,4197.76
1017210,2,7621.96
1017211,3,9075.84
1017212,4,7937.10
1017213,5,7604.44


In [53]:
sumbission[['Id', 'Sales']].to_csv('./answers/sumbission_6.csv', index=False)

Посмотрим важность признаков:

In [54]:
features = {}
for i, column in enumerate(columns):
    features[column] = rf.feature_importances_[i] 
features = sorted(features.items(), key=operator.itemgetter(1), reverse=True)
features

[('AverageCheck', 0.22253594217117495),
 ('Store', 0.18801805495213347),
 ('CompetitionDistance', 0.18127871797010187),
 ('Promo', 0.13560538249271681),
 ('DayOfWeek', 0.06747073417268222),
 ('DateDayOfYear', 0.05080641747655143),
 ('DateDayOfMonth', 0.036667002888354248),
 ('StoreType', 0.024308359216553652),
 ('Assortment', 0.024217235271058467),
 ('Promo2', 0.019721160566262178),
 ('DateWeekOfYear', 0.014874487360678788),
 ('DateYear', 0.014003255187930809),
 ('CompetitionIsOpen', 0.0072591403859985802),
 ('DateMonthOfYear', 0.0055869658833147739),
 ('SchoolHoliday', 0.0039350964815902476),
 ('Promo2Today', 0.0021581336071882849),
 ('StateHoliday', 0.001553913915709231),
 ('Open', 0.0)]

Лучший результат:<br>
Private Score 0.15533<br>
Public Score 0.13161